In [3]:
using CSV
using LowRankApprox

In [5]:
# make a large data set and save it to ../data/sample100000x100.txt
include("../code/julia/ash.jl");
include("../code/julia/mixSQP_time.jl");
include("../code/julia/makedata.jl");

In [6]:
L1 = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));
L2 = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));

In [7]:
include("../code/julia/mixSQP.jl");
x1 = mixSQP(L1, x = ones(size(L1,2))/size(L1,2); convtol = 1e-8,
                pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
                maxiter = 100, maxqpiter = 100,
                lowrank = "svd", seed = 1, verbose = true);

Running SQP algorithm with the following settings:
- 5000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-10
- partial SVD max. error = 1.98e-10
iter       objective -min(g+1) #nnz #qp
   1 2.95206790e+03 +1.81e-01   20   0
   2 2.55805274e+03 +2.20e-01    3  22
   3 2.37809188e+03 +4.08e-02    3   2
   4 2.34432187e+03 +5.72e-03    3   2
   5 2.33599066e+03 +2.59e-04    3   2
   6 2.33556636e+03 +4.81e-07    3   2
   7 2.33556553e+03 -9.52e-09    3   2
Optimization took 7 iterations and 0.1432 seconds.


In [8]:
x2 = mixSQP(L2, x = ones(size(L2,2))/size(L2,2); convtol = 1e-8,
                pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
                maxiter = 100, maxqpiter = 100,
                lowrank = "qr", seed = 1, verbose = true);

Running SQP algorithm with the following settings:
- 100000 x 100 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial QR tolerance  = 1.00e-10
- partial QR max. error = 2.69e-09
iter       objective -min(g+1) #nnz #qp
   1 5.03421516e+04 +2.90e-01  100   0
   2 5.19513613e+04 +1.06e-01    4 100
   3 4.83001092e+04 +1.57e-02    4  26
   4 4.76838683e+04 +4.20e-04    3  12
   5 4.76630722e+04 +4.85e-07    3   2
   6 4.76630469e+04 -9.29e-09    3   2
Optimization took 6 iterations and 1.3078 seconds.


## Session information

The section gives information about the computing environment used to generate the results contained in this
manuscript, including the version of Julia, Python and the Julia packages. 

In [9]:
Pkg.status("CSV")

 - CSV                           0.1.5


In [10]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
